## Step 11 가변 길이 인수(순전파 편)

In [8]:
import numpy as np

In [9]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
            
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # y.grad = np.array(1.0)생략 가능

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            x, y = f.input, f.output # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가한다.

In [10]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [16]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] #인수와 반환값 리스트로 변경
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] 

        for output in outputs:
            output.set_creator(self) 
        self.inputs = inputs
        self.outputs = outputs 
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
    def backward(self, gys):
        raise NotImplementedError()

In [17]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)#y 하나 가진 튜플 반환 , 있어야함

In [18]:
xs = [Variable(np.array(2)), Variable(np.array(3))] #리스트로 준비
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


In [20]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [21]:
def square(x):
    return Square()(x) 
def exp(x):
    return Exp()(x)

In [22]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [23]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
